In [0]:
import tensorflow as tf
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from tqdm import tqdm

def plot_mean_and_CI(mean, lb, ub, color_shading=None):
    plt.plot(mean, 'r', zorder=10)
    plt.fill_between(range(mean.shape[0]), ub, lb,
                     color=color_shading, alpha=0.4, zorder=15)

In [0]:
'''Class for environment simulation'''

class environment():
    
    '''environment initializer'''
    
    def __init__(self, init_s):
        
        '''init_s: tuple of initial parameters
        
           self.t: temperature of air
           self.vent_degree: vent. lebel
           self.t_h: temperature of heater
           self.t_out: temperature outside'''
        
        self.t, self.vent_degree, self.t_h, self.t_w, self.t_out = init_s
        
    '''one step of environment'''
        
    def step(self, a):
        
        '''a: tuple of actions
           heat: state of heater
           vent: state of vent.'''
        
        heat, vent = a
        
        '''update of env. parameters'''
        
        self.t = self.t - 0.01*(self.t_w - self.t) + 0.06*(self.t_h - self.t) + 0.04*vent - 0.004*self.t*vent - 0.0005*self.t ** 2
        self.t_h = self.t_h + 0.05 * heat * (70 - self.t_h) - 0.05 * (self.t_h - self.t)
        self.t_w = self.t_w + 0.05 * (self.t - self.t_w) - 0.05 * (self.t_out - self.t)
        self.t_out = self.t_out
        
        '''checking of boundaries'''
        
        if self.t<0:
            self.t = 0
        elif self.t>35:
            self.t = 35
            
        self.vent_degree = self.vent_degree - (1 - vent) + 3*vent
        
        if self.vent_degree<0:
            self.vent_degree = 0
        elif self.vent_degree>60:
            self.vent_degree = 60
            
        '''returns visible parameters of environment'''
            
        return self.t, self.vent_degree, self.t_out
      
    '''sample session with random control
    
       num_of_steps: number of time steps
       sigma: std deviation of additive noise
       '''
    def random_session(self, num_of_steps, sigma):
        '''initializing sets of env. parameters'''
        collection_a = []
        collection_t = []
        collection_noisy_t = []
        collection_t_out = []
        
        '''initial values of env. parameters'''
        t = self.t
        t_out = self.t_out
        noisy_t = t
        a = 3
        
        '''main loop'''
        
        for i in range(num_of_steps):
            collection_t.append(t)
            collection_noisy_t.append(noisy_t)
            collection_t_out.append(t_out)
            if 0.3 > np.random.rand():
                a = np.random.choice(4)
            collection_a.append(a)
            a_for_agent = np.unravel_index(a, (2, 2))
            t, _, t_out = self.step(a_for_agent)
            noisy_t = t + sigma*np.random.randn()
            
        '''returns set of t, set of t with additive gaussian noise, set of actions, set of outside temperatures'''
        return np.array(collection_t), np.array(collection_noisy_t), np.array(collection_a), np.array(collection_t_out)

In [0]:
'''generation of training set with random actions'''

'''evironment initialization'''
init_params = (20, 20, 60, 5, -5) #see description of the environment class
env = environment(init_params)

'''sample session with given length and std (total length of session: number_of_pieces * session_length)'''
number_of_pieces = 128
session_length = 300
std_of_noise = 0.25

s_length = number_of_pieces * session_length
t, noisy_t, a, t_out = env.random_session(s_length, std_of_noise)

'''reshaping dataset in appropriate for learning form'''
t_out = t_out.reshape((number_of_pieces, session_length))
t = t.reshape((number_of_pieces, session_length))
noisy_t = noisy_t.reshape((number_of_pieces, session_length))
a = a.reshape((number_of_pieces, session_length))

'''plotting of example'''
plt.ylabel('Temperature')
plt.xlabel('Time')
plt.title('Example')
plt.plot(t[10], 'b')

In [0]:
'''number of hidden neurons in GRU rnn'''
number_of_hid_units = 3


'''conv 1d layer, is needed for denoising'''
'''input_value: input tensor of shape (batch_size, width, channels_in)
   channels_in: number of input channels
   channels_out: number of output channels
   name: name
   activation: if True, apply ELU function to output, if False, keep output linear'''
def conv_1d_layer(input_value, channels_in, channels_out, name, activation=True, filter_size=16):
    with tf.name_scope(name):
        w = tf.get_variable(shape=(filter_size, channels_in, channels_out), name='w'+name, initializer=tf.initializers.he_uniform())
        b = tf.Variable(tf.zeros([channels_out]), name='b'+name)
        conv = tf.nn.conv1d(input_value, w, 1, "SAME", name='conv')
        if activation == True:
            act = tf.nn.elu(conv + b)
        else:
            act = conv + b
    return act

'''training of the model'''
'''sess: input tf. session
   states: set of states (temperatures)
   actions: set of actions
   t_out: set of outdoor temperatures
   number_of_steps: number of training steps'''
def train(sess, states, actions, t_out, number_of_steps=10000):
  
    '''losses set'''
    losses = []
    
    '''training loop'''
    for i in tqdm(range(number_of_steps)):
        l = sess.run([loss, train_step], feed_dict={actions_ph:actions, states_ph:states,\
                                                  eps_ph:np.random.randn(actions.shape[0], actions.shape[1]), t_out_ph:t_out, initial_state:np.random.randn(states.shape[0], number_of_hid_units)})[0]
        losses.append(l)
        
        '''plotting'''
        if i%1000==999:
        
            '''Denoised t and its std'''
            pred_t, sigma_t = t_denoising(sess, noisy_t[10], t_out[10], a[10])
            
            '''True dynamics and its noisy version'''
            true_t, true_noisy_t = t[10], noisy_t[10]

            '''plotting'''
            fig1 = plt.figure()
            plt.ylabel('Temperature')
            plt.xlabel('Time')
            plt.plot(true_noisy_t, 'black', zorder=1)
            plt.plot(true_t, 'green', zorder=5)
            plt.plot(pred_t, 'red', zorder=10)
            plt.legend(['Observed dynamics', 'Real dynamics', 'Learned dynamics'])
            plt.savefig(fname='fig')
            fig2 = plt.figure()
            plt.xlabel('Time')
            plt.ylabel('Temperature with 3*std')
            plot_mean_and_CI(pred_t, pred_t - 3*sigma_t, pred_t + 3*sigma_t, color_shading='red')
            fig3 = plt.figure()
            plt.ylabel('loss')
            plt.xlabel('epoch num')
            plt.yscale('log')
            plt.plot(losses, 'blue')
            plt.show()
            
            '''returns set of losses'''
    return losses
    
'''denoising of temperature'''
'''sess: input session
   noisy_t: observed temperature (with noise)
   t_out: outdoor temperature
   action: set of actions'''
def t_denoising(sess, noisy_t, t_out, action):
    data = sess.run([state_out], feed_dict={actions_ph:action.reshape((1, -1)), states_ph:noisy_t.reshape((1, -1)), t_out_ph:t_out.reshape((1, -1))})[0]
    pred_t = data[0, :, 0]
    sigma_t = np.exp(data[0, :, 1]/2)
    
    '''returns tuple of denoised temperature and std deviation'''
    return pred_t, sigma_t
  
'''FC net for hidden GRU state processing'''
'''Input: input tensor
   weights: dict of weights
   name: name'''
def dynamics_net(Input, weights, name):
    with tf.name_scope(name):
        h1 = tf.matmul(Input, weights['W1']) + weights['b1']
        h1 = tf.nn.elu(h1)
        return tf.matmul(h1, weights['W2']) + weights['b2']

In [6]:
'''Main tf. graph'''

'''length of one session'''
learning_rate = 0.003

'''min and max temperature'''
t_min = 0
t_max = 30


tf.reset_default_graph()

'''initial state of GRU rnn (hidden)'''
initial_state = tf.placeholder(shape=(None, number_of_hid_units), dtype=tf.float32, name='initial_state')

'''GRU net initialization (rnn_net is graph node which returns transformation after rnn)'''
with tf.name_scope('rnn_net'):
    Input = tf.keras.layers.Input(shape=(None, 6))
    Output = tf.keras.layers.CuDNNGRU(units=number_of_hid_units, return_sequences=True, return_state=True)(Input, initial_state=initial_state)
    rnn_net = tf.keras.Model(inputs=Input, outputs=Output)


'''below, main building blocks of main graph'''
    

#####log variances of normal noise in dynamical system and in observations respectively####
s = tf.constant(-5, dtype=tf.float32, name='log_variance_transition')
eta = tf.Variable(-5, dtype=tf.float32, name='log_variance_measurement')
###########################################################################################

####weights which parametrize FC net after GRU cell########################################
with tf.name_scope('weights'):
    weights = {}
    weights['W1'] = tf.get_variable(shape=(number_of_hid_units, 16), dtype=tf.float32, name='W1')
    weights['b1'] = tf.get_variable(shape=(16,), dtype=tf.float32, name='b1')
    weights['W2'] = tf.get_variable(shape=(16, 1), dtype=tf.float32, name='W2')
    weights['b2'] = tf.get_variable(shape=(1,), dtype=tf.float32, name='b2')
###########################################################################################

####placeholders for actions, tamp. (states_ph), temp. outside, auxiliary random variables#
actions_ph = tf.placeholder(shape=(None, session_length), dtype=tf.int32, name='actions_ph')
states_ph = tf.placeholder(shape=(None, session_length), dtype=tf.float32, name='states_ph')
eps_ph = tf.placeholder(shape=(None, session_length), dtype=tf.float32, name='eps_ph')
t_out_ph = tf.placeholder(shape=(None, session_length), dtype=tf.float32, name='t_out_ph')
###########################################################################################

####concatenation of states and actions####################################################
states_and_actions = tf.concat([tf.expand_dims(tf.cast(actions_ph, dtype=tf.float32),\
              axis=-1), tf.expand_dims(states_ph, axis=-1),\
                                tf.expand_dims(t_out_ph, axis=-1)], axis=-1, name='concat')
###########################################################################################


#####decoder (denoiser)####################################################################
conv1 = conv_1d_layer(states_and_actions, 3, 8, filter_size=8, name='conv1')
conv2 = conv_1d_layer(conv1, 8, 32, filter_size=8, name='conv2')
conv3 = conv_1d_layer(conv2, 32, 8, filter_size=8, name='conv3')
state_out = conv_1d_layer(conv3, 8, 2, filter_size=8, name='conv4', activation=False)
with tf.name_scope('separation'):
    #hidden variables
    mu = state_out[:, :, 0]
    #variance of hidden variables
    sigma = state_out[:, :, 1]
###########################################################################################

####one hot representation of actions######################################################
one_hot_action = tf.one_hot(actions_ph, axis=-1, depth=4, name='one_hot_action')
###########################################################################################

####data preparation before passing through dynamical map##################################
with tf.name_scope('true_state'):
    true_state = mu + tf.exp(sigma/2) * eps_ph

input_for_transformation = tf.concat([one_hot_action, tf.expand_dims(true_state, axis=-1), \
                                      tf.expand_dims(t_out_ph, axis=-1)]\
                                     , axis=-1, name='input_for_transformation')
###########################################################################################

####passing through dynamical map##########################################################
with tf.name_scope('dynamical_rnn'):
    
    rnn_out, _ = rnn_net(input_for_transformation)
    dyn_out = tf.reshape(dynamics_net(tf.reshape(rnn_out, \
                                shape=(-1, number_of_hid_units)), weights, 'fc'), shape=(-1, session_length))
    new_state = dyn_out + true_state
    
###########################################################################################

####shifting data in time##################################################################
with tf.name_scope('time_shift'):
    mu = mu[:, 1:]
    sigma = sigma[:, 1:]
    T = states_ph[:, 1:]
    eps = eps_ph[:, 1:]
    F = new_state[:, :-1]
###########################################################################################


####sigma term#############################################################################
with tf.name_scope('sigma_term'):
    sigma_term = tf.exp(sigma) * (1 / (2 * tf.exp(s)) + 1 / (2 * tf.exp(eta)))
###########################################################################################

####regularization term####################################################################
with tf.name_scope('regularization_term'):
    regularization_term = tf.pow(mu - F, 2) / (2 * tf.exp(s))
###########################################################################################

####log sigma term#########################################################################
with tf.name_scope('log_sigma_term'):
    log_sigma_term = -sigma/2
###########################################################################################

####regression term########################################################################
with tf.name_scope('regression_term'):
    regression_term = tf.pow(T - mu, 2) / (2 * tf.exp(eta))
###########################################################################################

####loss function##########################################################################
with tf.name_scope('loss'):
    loss = tf.reduce_mean(sigma_term + regularization_term + \
                          log_sigma_term + regression_term + s/2 + eta/2)
###########################################################################################

####training###############################################################################
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################

'''auxiliary piece of graph, which is responsible for extracting hidden state of GRU via passing
previous visible states to GRU'''

################placeholders for actions, temp. (hidden_states_ph) and outside temp.#######
hidden_actions_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='hidden_actions_ph')
hidden_states_ph = tf.placeholder(shape=(None, None), dtype=tf.float32, name='hidden_states_ph')
hidden_t_out_ph = tf.placeholder(shape=(None, None), dtype=tf.float32, name='hidden_t_out_ph')
###########################################################################################

######################one hot representation of the actions################################
hidden_one_hot_action = tf.one_hot(hidden_actions_ph, axis=-1, depth=4, name='hidden_one_hot_action')
###########################################################################################

#####################preparing data for passing through dynamical net######################
hidden_input_for_transformation = tf.concat([hidden_one_hot_action, tf.expand_dims(hidden_states_ph, axis=-1), \
                                      tf.expand_dims(hidden_t_out_ph, axis=-1)]\
                                     , axis=-1, name='hidden_input_for_transformation')
###########################################################################################


##########################hidden state of GRU (new_hidden_state)###########################
_, new_hidden_state = rnn_net(hidden_input_for_transformation)
###########################################################################################

###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################

'''auxiliary piece of graph, which is responsible for prediction of dynamics'''

################placeholders for actions, temp. (hidden_states_ph) and outside temp.#######
pred_actions_ph = tf.placeholder(shape=(None, 1), dtype=tf.int32, name='pred_actions_ph')
pred_states_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='pred_states_ph')
pred_t_out_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='pred_t_out_ph')
###########################################################################################

######################one hot representation of the actions################################
pred_one_hot_action = tf.one_hot(pred_actions_ph, axis=-1, depth=4, name='pred_one_hot_action')
###########################################################################################

#####################preparing data for passing through dynamical net######################
pred_input_for_transformation = tf.concat([pred_one_hot_action, tf.expand_dims(pred_states_ph, axis=-1), \
                                      tf.expand_dims(pred_t_out_ph, axis=-1)]\
                                     , axis=-1, name='pred_input_for_transformation')
###########################################################################################

########################next state prediction (new_pred)###################################
with tf.name_scope('pred_rnn'):
    
    pred_out, pred_state = rnn_net(pred_input_for_transformation)
    pred_dyn = tf.reshape(dynamics_net(tf.reshape(pred_out, \
                                shape=(-1, number_of_hid_units)), weights, 'fc_pred'), shape=(-1, 1))
    new_pred = pred_dyn + pred_states_ph
###########################################################################################

'''saver initialization'''
saver = tf.train.Saver()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
'''Training'''
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
losses = train(sess, noisy_t, a, t_out, number_of_steps=100000)
save_path = saver.save(sess, "/model.ckpt")

In [0]:
'''Plotting an example'''

'''Denoised t and its std'''
pred_t, sigma_t = t_denoising(sess, noisy_t[8], t_out[8], a[8])
'''True dynamics and its noisy version'''
true_t, true_noisy_t = t[8], noisy_t[8]

'''plotting'''
fig1 = plt.figure()
plt.ylabel('Temperature')
plt.xlabel('Time')
plt.plot(true_noisy_t, 'black', zorder=1)
plt.plot(true_t, 'green', zorder=5)
plt.plot(pred_t, 'red', zorder=10)
plt.legend(['Observed dynamics', 'Real dynamics', 'Learned dynamics'])
plt.savefig(fname='fig')
fig2 = plt.figure()
plt.xlabel('Time')
plt.ylabel('Temperature with 3*std')
plot_mean_and_CI(pred_t, pred_t - 3*sigma_t, pred_t + 3*sigma_t, color_shading='red')
fig3 = plt.figure()
plt.ylabel('loss')
plt.xlabel('epoch num')
plt.yscale('log')
plt.plot(losses, 'blue')
plt.show()

In [0]:
'''extraction of the hidden state from historical data'''
'''t: temperatures
   actions: actions
   t_out: outdoor temperature'''
def extract_hidden_state(t, actions, t_out):
    '''returns hidden state of GRU'''
    return sess.run(new_hidden_state, \
           feed_dict={hidden_actions_ph:actions,\
           hidden_states_ph:t,\
           hidden_t_out_ph:t_out,\
           initial_state:np.random.randn(t.shape[0], number_of_hid_units)})
  
'''prediction of dynamics'''
'''t: current temperature
   action: current action
   t_out: current outdoor temperature
   in_rnn_state: current hidden state of GRU'''
def dyn_predict(t, action, t_out, in_rnn_state):
    '''returns tuple of (new temperature, new hidden state of GRU)'''
    return sess.run([new_pred, pred_state],\
           feed_dict={pred_actions_ph:action.reshape((1, 1)),\
           pred_states_ph:t.reshape((1, 1)),\
           pred_t_out_ph:t_out.reshape((1, 1)),\
           initial_state:in_rnn_state})

In [0]:
'''piece of code for testing of prediction'''

'''initializing env.'''
env = environment((20, 20, 60, 5, -5))

'''sample true dynamics'''
t, noisy_t, a, t_out = env.random_session(600, 0.25)

'''historical data for extracting GRU hidden state'''
t_hist = t[:300].reshape((1, -1))
a_hist = a[:300].reshape((1, -1))
t_out_hist = t_out[:300].reshape((1, -1))

'''future data, we are going to predict'''
t_future = t[300:]
a_future = a[300:]
t_out_future = t_out[300:]

'''current temperature of prediction'''
s = np.array(t[300])

'''extraction of hidden GRU state using historical data'''
in_rnn_state = extract_hidden_state(t_hist, a_hist, t_out_hist)

'''prediction loop'''

'''initializing t collection'''
t_test = []
for i in range(300):
  
    '''update temperature (s) and hidden GRU state'''
    s, in_rnn_state = dyn_predict(s, a_future[i], t_out_future[i], in_rnn_state)
    
    '''adding to t collection'''
    t_test.append(s)
    
t_test = np.array(t_test).flatten()

In [0]:
'''plotting comparison of true future dynamics and prediction'''
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.plot(t_test, 'blue')
plt.plot(t[300:], 'red')
plt.legend(['True dynamics', 'Learned dynamics'])